# Proyecto 1 - Mineria de datos
## Explorando patrones de robos por edad de las victimas en Guatemala

### Autores:
#### Andrés de la Roca
#### Jun Woo Lee

### Analisis exploratorio

In [ ]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import numpy as np

from quickda.explore_data import *
from quickda.clean_data import *
from quickda.explore_numeric import *
from quickda.explore_categoric import *
from quickda.explore_numeric_categoric import *
from quickda.explore_time_series import *

#### Importar la data

### Analisis por clustering